# CELLO Mk 3.0, EST. 2023-3-3


## COMPONENT #2:
## Restricted Graph Conversion to Python netlist object

SAVES the Netlist JSON file produced by YOSYS as an OBJECT such that we can map and evaluates parts to it

In [1]:
import json

In [2]:
in_path = '../../IO/inputs'
out_path = '../../IO/temp_folder'

In [3]:
# ENTRY POINT
netlist_name = 'g92_boolean'

In [4]:
netlist_path = out_path + '/' + netlist_name + '.json'

In [5]:
netlist_file = open(netlist_path, 'r')
netlist = json.load(netlist_file)
'''
!important: netlist has in the json format generated by YOSYS
TODO: Add conversion from other netlist types (such as STL)
'''

'\n!important: netlist has in the json format generated by YOSYS\nTODO: Add conversion from other netlist types (such as STL)\n'

In [6]:
print(len(netlist))
# 2 is right

2


In [7]:
print(json.dumps(netlist, indent=4))

{
    "creator": "Yosys 0.25 (git sha1 e02b7f64bc7, clang 14.0.0 -fPIC -Os)",
    "modules": {
        "Main": {
            "attributes": {
                "top": "00000000000000000000000000000001",
                "src": "../../IO/inputs/g92_boolean.v:1.1-14.12"
            },
            "ports": {
                "t1": {
                    "direction": "input",
                    "bits": [
                        2
                    ]
                },
                "in91": {
                    "direction": "input",
                    "bits": [
                        3
                    ]
                },
                "out108": {
                    "direction": "output",
                    "bits": [
                        4
                    ]
                },
                "out98": {
                    "direction": "output",
                    "bits": [
                        5
                    ]
                }
            },
            "cells":

In [8]:
circuit_name = list(netlist["modules"].keys())[0]
print(circuit_name)

Main


In [9]:
components = []
# dev res/s
for m in netlist["modules"][circuit_name]:
    components.append(m)
    print(m)

attributes
ports
cells
netnames


In [10]:
if 'parameter_default_values' in components:
    print("we've got an extra component")
else:
    print('we\'re good')

we're good


In [11]:
'''
_sort_gates(self, cells)

'''

cls = netlist["modules"][circuit_name]['cells']
#print(cls)
# print(type(cls))

for c in cls.keys():
    partition = list(c.split('$'))
    gate_id = partition[-1]
    gate_type = cls[c]['type']
#     print(gate_id + ' ' + gate_type + ' ' + partition[1])
    gate = {gate_id : 
            {
             'type' : gate_type,
             'inputs' : {},
             'output' : {},
            }
           }
#     print(gate)
#     print(json.dumps(gate, indent=4))
#     print()
    dirns = cls[c]['port_directions']
    ctns = cls[c]['connections']
    inputs = []
    outputs = []
    for d in dirns.keys():
        inout = dirns[d]
        if inout == 'input':
            inputs.append(d)
        else:
            outputs.append(d)
    gate_inputs = []
    gate_outputs = []
    for c in ctns.keys():
        c_nodes = ctns[c]
        if c in inputs:
            gate_inputs.append((c, c_nodes))
        else:
            gate_outputs.append((c, c_nodes))
#     gate[gate_id]['inputs'] = gate_inputs
#     gate[gate_id]['output'] = gate_outputs
#     print(gate_inputs)
#     print(gate_outputs)
    for tup in gate_inputs:
        (node_name, edge_nos) = tup
        try:
            gate[gate_id]['inputs'][node_name] += edge_nos
        except Exception as e:
            gate[gate_id]['inputs'][node_name] = edge_nos
    for tup in gate_outputs:
        (node_name, edge_nos) = tup
        try:
            gate[gate_id]['output'][node_name] += edge_nos
        except Exception as e:
            gate[gate_id]['output'][node_name] = edge_nos
    print(json.dumps(gate, indent=4))
#         print(d + ' ' + str(dirns[d]))
#     for n in ctns.keys():

{
    "100": {
        "type": "$_NOR_",
        "inputs": {
            "A": [
                5
            ],
            "B": [
                6
            ]
        },
        "output": {
            "Y": [
                7
            ]
        }
    }
}
{
    "101": {
        "type": "$_NOT_",
        "inputs": {
            "A": [
                7
            ]
        },
        "output": {
            "Y": [
                4
            ]
        }
    }
}
{
    "96": {
        "type": "$_NOT_",
        "inputs": {
            "A": [
                2
            ]
        },
        "output": {
            "Y": [
                8
            ]
        }
    }
}
{
    "97": {
        "type": "$_NOT_",
        "inputs": {
            "A": [
                3
            ]
        },
        "output": {
            "Y": [
                9
            ]
        }
    }
}
{
    "98": {
        "type": "$_NOR_",
        "inputs": {
            "A": [
                8
     

In [12]:
ports = netlist["modules"][circuit_name]['ports']

in_nodes = []
out_nodes = []
for p in ports.keys():
    node_name = p
    direction = ports[p]['direction']
    bits = ports[p]['bits']
    node = (node_name, bits)
    if direction == 'input':
        in_nodes.append(node)
    elif direction == 'output':
        out_nodes.append(node)
    else:
        raise ValueError('Invalid [in/out]put node') 
print("inputs: " + str(in_nodes))
print("outputs: " + str(out_nodes))

inputs: [('t1', [2]), ('in91', [3])]
outputs: [('out108', [4]), ('out98', [5])]


In [13]:
class Netlist:
    def __init__(self, netlistJSON):
        net_main = netlistJSON['modules']
        self.name = list(net_main.keys())[0]
        ports = net_main[self.name]['ports']
        cells = net_main[self.name]['cells']
        edges = net_main[self.name]['netnames']
        i, o = self.__sort_nodes(ports)
        self.inputs = i
        self.outputs = o
        self.gates = self.__sort_gates(cells)
        self.edges = self.__check_edges(edges)
        
    def __sort_nodes(self, ports):
        in_nodes = []
        out_nodes = []
        for p in ports.keys():
            node_name = p
            direction = ports[p]['direction']
            bits = ports[p]['bits']
            node = (node_name, bits)
            if direction == 'input':
                in_nodes.append(node)
            elif direction == 'output':
                out_nodes.append(node)
            else:
                raise ValueError('Invalid [in/out]put node') 
        return (in_nodes, out_nodes)
    
    def __sort_gates(self, cells):
        gates = []
        for c in cells.keys():
            partition = list(c.split('$'))
            gate_id = partition[-1]
            gate_type = cells[c]['type']
            gate = {gate_id : 
                    {
                     'type' : gate_type,
                     'inputs' : {},
                     'output' : {},
                    }
                   }
            dirns = cells[c]['port_directions']
            ctns = cells[c]['connections']
            inputs = []
            outputs = []
            for d in dirns.keys():
                inout = dirns[d]
                if inout == 'input':
                    inputs.append(d)
                else:
                    outputs.append(d)
            gate_inputs = []
            gate_outputs = []
            for c in ctns.keys():
                c_nodes = ctns[c]
                if c in inputs:
                    gate_inputs.append((c, c_nodes))
                else:
                    gate_outputs.append((c, c_nodes))
            for tup in gate_inputs:
                (node_name, edge_nos) = tup
                try:
                    gate[gate_id]['inputs'][node_name] += edge_nos
                except Exception as e:
                    gate[gate_id]['inputs'][node_name] = edge_nos
            for tup in gate_outputs:
                (node_name, edge_nos) = tup
                try:
                    gate[gate_id]['output'][node_name] += edge_nos
                except Exception as e:
                    gate[gate_id]['output'][node_name] = edge_nos
            #print(json.dumps(gate, indent=4))
            gates.append(gate)
        return gates
    
    def __str__(self):
        return (f"{self.name}: with \n"
                f"{len(self.inputs)} inputs,\n"
                f"{len(self.outputs)} ouputs.")
    
    def __check_edges(self, edges):
        for n in edges:
            pass
        return 9

In [14]:
testNet = Netlist(netlist)

In [15]:
print(testNet.inputs)

[('t1', [2]), ('in91', [3])]


In [16]:
print(testNet.outputs)

[('out108', [4]), ('out98', [5])]


In [17]:
print(len(testNet.gates))

6


In [18]:
print(testNet)

Main: with 
2 inputs,
2 ouputs.
